In [1]:
import json


data_path = "../data/VLSP 2025 - MLQA-TSR Data Release/train_data/vlsp_2025_train.json"
with open(data_path, "r") as f:
    data = json.load(f)

print(data[0:10])



[{'id': 'train_1', 'image_id': 'train_1_3', 'question': 'Biển báo cấm xe khách trên 29 chỗ được áp dụng trong các khoảng thời gian nào? ', 'relevant_articles': [{'law_id': 'QCVN 41:2024/BGTVT', 'article_id': '22'}], 'question_type': 'Multiple choice', 'choices': {'A': 'Từ 6:30 đến 8:00 và từ 16:30 đến 18:30; ngoài các khoảng thời gian này không được phép lưu thông.', 'B': 'Từ 6:30 đến 8:00 và từ 16:30 đến 18:30; ngoài các khoảng thời gian này được phép lưu thông.', 'C': 'Cấm lưu thông cả ngày.', 'D': 'D. Không cấm xe khách trên 29 chỗ lưu thông.'}, 'answer': 'B'}, {'id': 'train_2', 'image_id': 'train_1_3', 'question': 'Những loại phương tiện nào bị cấm trên đoạn đường này', 'relevant_articles': [{'law_id': 'QCVN 41:2024/BGTVT', 'article_id': '26'}, {'law_id': 'QCVN 41:2024/BGTVT', 'article_id': '14'}], 'question_type': 'Multiple choice', 'choices': {'A': 'Xe khách trên 29 chỗ', 'B': 'ô tô con', 'C': 'Xe máy', 'D': 'Xe đạp'}, 'answer': 'A'}, {'id': 'train_3', 'image_id': 't

In [2]:
for record in data:
    if record["question_type"] == "Yes/No":
        answer = record.pop("answer")
        record["choices"] = None
        record["answer"] = answer
    file_name = f"{record["image_id"]}.jpg"
    record["file_name"] = file_name

In [3]:
print(data[0].keys())

dict_keys(['id', 'image_id', 'question', 'relevant_articles', 'question_type', 'choices', 'answer', 'file_name'])


In [4]:
import csv
df = open("../data/metadata.csv", "w")
writer = csv.writer(df)
for idx, data_row in enumerate(data):
    if idx==0:
        header = data_row.keys()
        writer.writerow(header)
    writer.writerow(data_row.values())
df.close()

In [5]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="../data/VLSP 2025 - MLQA-TSR Data Release/train_data/train_images")

/home/nhhsag12/Documents/Multimodal Machine Learning/VLSP_2025/vlsp_2025_multimodal_rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 530 examples [00:00, 5354.22 examples/s]


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'image_id', 'question', 'relevant_articles', 'question_type', 'choices', 'answer', 'image'],
        num_rows: 530
    })
})

In [7]:
dataset.push_to_hub("nhhsag12/vlsp_2025_multimodal_rag", private=True)


Map: 100%|██████████| 530/530 [00:00<00:00, 635.82 examples/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:10<00:00, 10.25s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/nhhsag12/vlsp_2025_multimodal_rag/commit/65c7f4966ad691ff792709005c8caa1ec4bab7c2', commit_message='Upload dataset', commit_description='', oid='65c7f4966ad691ff792709005c8caa1ec4bab7c2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/nhhsag12/vlsp_2025_multimodal_rag', endpoint='https://huggingface.co', repo_type='dataset', repo_id='nhhsag12/vlsp_2025_multimodal_rag'), pr_revision=None, pr_num=None)

In [1]:
import torch
from PIL import Image
from src.reranker.cross_encoder import CrossEncoder
from src.multimodal_retriever.retriever_v2 import RetrieverV2

reranker = RetrieverV2(pretrained_model_path="../src/multimodal_retriever/pretrained_model/Visualized_m3.pth").to("cpu")


/home/nhhsag12/Documents/Multimodal Machine Learning/VLSP_2025/vlsp_2025_multimodal_rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nhhsag12/Documents/Multimodal Machine Learning/VLSP_2025/vlsp_2025_multimodal_rag/.venv/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
print(reranker.device)

AttributeError: 'RetrieverV2' object has no attribute 'device'

In [4]:
image_path_1 = "../data/VLSP 2025 - MLQA-TSR Data Release/public_test/public_test_images/public_test_images/public_test_2_1.jpg"
image_path_2 = "../data/VLSP 2025 - MLQA-TSR Data Release/public_test/public_test_images/public_test_images/public_test_2_2.jpg"
image_1 = Image.open(image_path_1)
image_2 = Image.open(image_path_1)
preprocessed_images = reranker.preprocess_train(image_1)
preprocessed_images

tensor([[[-1.4273, -1.3981, -1.3981,  ..., -1.6025, -1.2521, -1.5003],
         [-1.3835, -1.3689, -1.3981,  ..., -1.2959, -1.3251, -1.5587],
         [-1.4273, -1.3835, -1.3981,  ..., -1.4127, -1.4273, -1.5879],
         ...,
         [ 0.4413,  0.4121,  0.4705,  ...,  0.7041,  0.6749,  0.6749],
         [ 0.3829,  0.4121,  0.5289,  ...,  0.6603,  0.7333,  0.7333],
         [ 0.5435,  0.4413,  0.5435,  ...,  0.5873,  0.5435,  0.7041]],

        [[-1.4219, -1.3919, -1.3919,  ..., -1.5720, -1.2418, -1.4219],
         [-1.3769, -1.3619, -1.3919,  ..., -1.2869, -1.3169, -1.4820],
         [-1.4219, -1.3769, -1.3919,  ..., -1.4519, -1.4069, -1.5270],
         ...,
         [ 0.4841,  0.4691,  0.5291,  ...,  0.6942,  0.6642,  0.6642],
         [ 0.4240,  0.4691,  0.5891,  ...,  0.6491,  0.7242,  0.7242],
         [ 0.5891,  0.4991,  0.6041,  ...,  0.5741,  0.5291,  0.6942]],

        [[-1.2385, -1.2100, -1.2100,  ..., -1.3380, -1.1247, -1.3238],
         [-1.2100, -1.1816, -1.2100,  ..., -1

In [9]:
image_path_1 = "../data/VLSP 2025 - MLQA-TSR Data Release/public_test/public_test_images/public_test_images/public_test_2_1.jpg"
image_path_2 = "../data/VLSP 2025 - MLQA-TSR Data Release/public_test/public_test_images/public_test_images/public_test_2_2.jpg"
image_1 = Image.open(image_path_1)
image_2 = Image.open(image_path_1)
processed_image_1 = reranker.preprocess(image_1).unsqueeze(0)
processed_image_2 = reranker.preprocess(image_2).unsqueeze(0)
print(processed_image_1.shape)
processed_images = torch.cat([processed_image_1, processed_image_2], dim=0)
print(processed_images)
print(processed_images.shape)

image_embedding = reranker.encode_image(processed_images)
print(image_embedding)
print(image_embedding.shape)

torch.Size([1, 3, 224, 224])
tensor([[[[-0.9310, -0.9164, -0.7996,  ..., -1.5295, -0.6974, -1.4565],
          [-0.9018, -0.9310, -0.7850,  ..., -1.5879, -1.5149, -1.5587],
          [-0.7996, -0.8434, -0.8288,  ..., -1.6171, -1.6901, -0.8580],
          ...,
          [ 0.4997,  0.5143,  0.5435,  ...,  0.8063,  0.6019,  0.5581],
          [ 0.5581,  0.5727,  0.5435,  ...,  0.7479,  0.6895,  0.6311],
          [ 0.4997,  0.4851,  0.4851,  ...,  0.5873,  0.6165,  0.7187]],

         [[-0.8366, -0.8366, -0.7166,  ..., -1.4369, -0.5965, -1.3769],
          [-0.8066, -0.8516, -0.7016,  ..., -1.5120, -1.4369, -1.4820],
          [-0.6715, -0.7466, -0.7466,  ..., -1.5270, -1.6170, -0.7466],
          ...,
          [ 0.5591,  0.5741,  0.6041,  ...,  0.8442,  0.6491,  0.6041],
          [ 0.6191,  0.6341,  0.6041,  ...,  0.7542,  0.6942,  0.6341],
          [ 0.5591,  0.5441,  0.5441,  ...,  0.5741,  0.6191,  0.7242]],

         [[-0.7692, -0.7408, -0.6128,  ..., -1.2243, -0.4279, -1.1532],
 

In [10]:
text_1 = "Xin chào tôi tên là Nguyễn Huy Hoàng"
text_2 = "Xin chào tôi ến từ Việt Nam"
text_embedding_1 = reranker.encode_text(text_1)
print(text_embedding_1.shape)
print(text_embedding_1)

torch.Size([1, 1024])
tensor([[-0.0285,  0.0262, -0.0354,  ...,  0.0166, -0.0540,  0.0173]],
       device='cuda:0', grad_fn=<DivBackward0>)


In [11]:
text_1 = "Xin chào tôi tên là Nguyễn Huy Hoàng"
text_2 = "Xin chào tôi ến từ Việt Nam"
texts = [text_1, text_2]
text_embedding_1 = reranker.encode_text(text_1)
text_embeddings = reranker.encode_text(texts)
# text_embeddings = torch.cat([text_embedding_1, text_embedding_2], dim=0)
print(text_embeddings.shape)
print(text_embeddings)

torch.Size([2, 1024])
tensor([[-0.0081,  0.0150, -0.0645,  ...,  0.0069, -0.0360,  0.0149],
        [-0.0243,  0.0007, -0.0634,  ..., -0.0040, -0.0588, -0.0341]],
       device='cuda:0', grad_fn=<DivBackward0>)


In [12]:
query_texts = [text_1, text_2]
doc_texts = ["Nguyễn Huy Hoàng là một cái tên phổ biến ở Việt Nam", "Việt Nam là một đất nước xinh đẹp và có bề dày lịch sử"]
query_images = processed_images.to(reranker.device)

In [13]:
relevant_score = reranker(query_texts, query_images)
print(relevant_score)

tensor([[ 0.1144, -1.4418, -0.3835,  ...,  0.1487, -0.1129,  1.4252],
        [-0.1711, -1.4158, -0.4482,  ..., -0.0931, -0.4035,  1.1276]],
       device='cuda:0', grad_fn=<NativeLayerNormBackward0>)


In [14]:
print(relevant_score.shape)

torch.Size([2, 1024])


In [15]:
mm_embeddings = reranker.encode_mm(query_texts, query_images)

In [16]:
mm_embeddings.shape

torch.Size([2, 1024])

In [14]:
torch.diagonal(relevant_score)

tensor([0.5498, 0.4582], device='cuda:0', grad_fn=<DiagonalBackward0>)